# Tinder para empresas

## Imports

In [1]:
# Imports
import numpy as np
import pandas as pd
from scipy.spatial import distance
from random import choice
from beautifultable import BeautifulTable


# from translate_csv_functions import translate_csv

# from sklearn.preprocessing import MinMaxScaler
# pip3 install pandas ipython jupyter

## Preprocesado csv de las encuestas Web

Descargar como CSV cada uno de los archivos de las respuestas en la carpeta ```./encuestas``` con el nombre por defecto que proporciona google: 
```
Forms Students: 
https://docs.google.com/spreadsheets/d/1tESGSKhoOWekvYnvMypd6t1eFXN_av8F60wyxPkbUb8/edit#gid=1960323408

Forms Company: 
https://docs.google.com/spreadsheets/d/1AHnY6-E9w__BNcWqiS2rkqYbNlmqaU6nPFiPHDDKRfI/edit#gid=730113545

Encuestas Students: 
https://docs.google.com/forms/d/1w85z6lqZdQfByTMiefqUcth16-F9NnLHuZuh9CVHxyU/edit#responses

Encuestas Company: 
https://docs.google.com/forms/d/1JOjjmV1W-ay7h00wVtpcQzD6FHNJU8Qg6RRz1ptDN4Q/edit
```
## Actual BNC web
```
Students (tab 11APR): https://docs.google.com/spreadsheets/d/128bKUO5In9JPMMs9JTLAhL-KxnmhiB3L2qSdyGKdxnY/edit#gid=1970037490

Companies (tab 11APR): https://docs.google.com/spreadsheets/d/1r6zryUQRl3X_4sbcfzpUrNIuT8lBAxVC7Ov7tItmfhc/edit#gid=1943728679
```
## MAdrid web
```
https://docs.google.com/spreadsheets/d/1bk_R2l83sWOd8VQyQV3R_n2Ikzx69QrMpfyvg6PpYms/edit?ts=5cb6d4ad#gid=177161654
```

### 1. Companies

In [2]:
# Funciones translate CSV

DIC_COMPANIES = {
    "What is the name of your company?": "name", 
    'Does your company have specific language requirements for this position? [English]': 'english', 
    'Does your company have specific language requirements for this position? [Spanish]': 'spanish', 
    'Does your company have specific language requirements for this position? [Portuguese]': 'portuguese', 
    'Does your company have specific language requirements for this position? [French]': 'french', 
    'Does your company have specific language requirements for this position? [Dutch]': 'dutch', 
    'Does your company have specific language requirements for this position? [Catalan]': 'catalan', 
    'Where will this position be located?': 'location', 
    'Regarding remote working, how many days a week could the employee work off site?': 'offsite', 
    'On a scale from 1 to 10, how much time will the employee be spending working on back end and front end?': 'position', 
    'Aside from the ones covered in our curriculum (MERN Stack), are there additional specific hard-skills you would value in an employee? [Java]': 'java', 
    'Aside from the ones covered in our curriculum (MERN Stack), are there additional specific hard-skills you would value in an employee? [C# / ASP.net]': 'caspnet',
    'Aside from the ones covered in our curriculum (MERN Stack), are there additional specific hard-skills you would value in an employee? [Python]': 'python',
    'Aside from the ones covered in our curriculum (MERN Stack), are there additional specific hard-skills you would value in an employee? [PHP]': 'php',
    'Aside from the ones covered in our curriculum (MERN Stack), are there additional specific hard-skills you would value in an employee? [SQL]': 'sql',
    'Aside from the ones covered in our curriculum (MERN Stack), are there additional specific hard-skills you would value in an employee? [Angular (Typescript generation)]': 'angular',
    'Aside from the ones covered in our curriculum (MERN Stack), are there additional specific hard-skills you would value in an employee? [Vue]': 'vue',
    'Aside from the ones covered in our curriculum (MERN Stack), are there additional specific hard-skills you would value in an employee? [Firebase]': 'firebase',
    'Aside from the ones covered in our curriculum (MERN Stack), are there additional specific hard-skills you would value in an employee? [AWS]': 'aws',
    'Aside from the ones covered in our curriculum (MERN Stack), are there additional specific hard-skills you would value in an employee? [Docker / Kubernetes]': 'dockerkubernetes',
    'Aside from the ones covered in our curriculum (MERN Stack), are there additional specific hard-skills you would value in an employee? [Design Tools (Photoshop, Illustrator...)]': 'design', 
    'From this list, please rank these 3 soft skills in order of importance for this position [Motivation and ability to overcome problems: employee has a passionate, can-do attitude and proactively looks for solutions to every problem they encounter]': 'motivation',
    'From this list, please rank these 3 soft skills in order of importance for this position [Coachability: Employee is receptive and actively listens and acts on feedback received]': 'coachability',
    'From this list, please rank these 3 soft skills in order of importance for this position [Teamwork: Employee is able to receive and provide value in a team environment]\r': 'teamwork' # El \r del final es para que sirva de terminador
    }

LANGUAGES = ['english', 'spanish', 'portuguese', 'french', 'dutch', 'catalan']

HARDSKILLS = ['java', 'caspnet', 'python', 'php', 'sql', 'angular', 
              'vue', 'firebase', 'aws', 'dockerkubernetes', 'design']

def translate_language(x): 
    DIC_RESP_LANGUAGE = {
        # Company
        "No need": 1, 
        "Must be able to read basic documentation": 2, 
        "Must be able to read documentation and have an informal conversation": 3, 
        "Must be able to read documentation and communicate in a meeting": 4, 
        "Can work in full capacity in this language": 5, 
        # Students same
        "No Experience": 1, 
        "I'm able to read basic documentation": 2, 
        "I'm able to read documentation and have an informal conversation": 3, 
        "Able to read documentation and communicate in a meeting": 4, 
        "Can work in full capacity in this language": 5
    }
    # return 0 if x not in DIC_RESP_LANGUAGE.keys() else DIC_RESP_LANGUAGE[x] 
    return DIC_RESP_LANGUAGE.get(x, 0) # 0 default

def translate_location(x): 
    DIC_RESP_LOCATION = {
        # Company
        "In the city where the campus is located": 1, 
        "In the country where the campus is located": 2, 
        "In a city in another country": 3, 
        # Student
        "In the city where the campus is located": 1, 
        "Anywhere in the country where the campus is located": 2, 
        "Anywhere in the world": 3
    }
    # return 0 if x not in DIC_RESP_LOCATION.keys() else DIC_RESP_LOCATION[x] 
    return DIC_RESP_LOCATION.get(x, 0) # 0 default


def translate_offsite(x): 
    DIC_RESP_OFFSITE = {
        # Company
        "100% office-based job. We're not keen on remote working.": 1, 
        "1-2 Days a week": 2, 
        "3+ Days. It's up to the employee": 3, 
        "100% remote-based job. No physical office.": 4, 
        # Student
        "100% office-based job.": 1, 
        "1-2 Days a week": 2, 
        "3+ Days": 3, 
        "100% remote-based job. No physical office.": 4
    }
    # return 0 if x not in DIC_RESP_OFFSITE.keys() else DIC_RESP_OFFSITE[x] 
    return DIC_RESP_OFFSITE.get(x, 0) # 0 default

def translate_position(x): 
    try: 
        return int(x)
    except: 
        return 0

def translate_hardskills(x): 
    DIC_RESP_HARDSKILLS = {
        # Company
        "No Need": 1, 
        "Nice to have: They should have a basic knowledge": 2, 
        "Must have: Must be able to work with the technology on a regular basis": 3,   
        # Student
        "No Experience": 1, 
        "Basic Knowledge: I've played around with it": 2, 
        "Advanced Knowledge: I'm comfortable working with it": 3
    }
    # return 0 if x not in DIC_RESP_HARDSKILLS.keys() else DIC_RESP_HARDSKILLS[x] 
    return DIC_RESP_HARDSKILLS.get(x, 0) # 0 default

   

def translate_csv(path, dic): 
    # open and setup columns
    # df = pd.read_csv(path, sep=',', index_col=0, lineterminator='\n') # para cuando tiene timestamp
    df = pd.read_csv(path, sep=',', lineterminator='\n')
    # display(df.head())
    df.rename(index=str, columns=dic, inplace=True)
    print(df.columns)
    df.index = df.name
    df.drop(columns=['name'], axis=1, inplace=True)
    
    print(df.columns)
    
    # languages
    for language in LANGUAGES: 
        df[language] = df[language].apply(translate_language)
    
    # location
    df['location'] = df['location'].apply(translate_location)    
    
    # offsite
    df['offsite'] = df['offsite'].apply(translate_offsite)
    
    # position
    df['position'] = df['position'].apply(translate_position)
    
    # hardskills
    for hardskill in HARDSKILLS: 
        df[hardskill] = df[hardskill].apply(translate_hardskills)
    
    return df

In [3]:
# COMPANIES_ENCUESTAS_CSV = './encuestas/Web - Ironhack Matchmaking Company Form (Responses) - Form Responses 1.csv'

# BNC web-day1
# COMPANIES_ENCUESTAS_CSV = './encuestas/Web - Ironhack Matchmaking Company Form (Responses) - 11APRWEB.csv'

# BNC web-day2
# COMPANIES_ENCUESTAS_CSV = './encuestas/Web - Ironhack Matchmaking Company Form (Responses) - 12APRWEB.csv'

# Madrid WEB company
COMPANIES_ENCUESTAS_CSV = './encuestas/HIRING FAIR APR 2019 MATCHMAKING - WD COMPANIES.csv'

companies_enc = translate_csv(path=COMPANIES_ENCUESTAS_CSV, dic=DIC_COMPANIES)
display(companies_enc.head(5))
# display(companies_enc.dtypes)
COMPANIES_CLEAN_CSV = './encuestas/companies.csv'
companies_enc.to_csv(COMPANIES_CLEAN_CSV, index=True)

# companies_enc.columns

Index(['name', 'english', 'spanish', 'portuguese', 'french', 'dutch',
       'catalan', 'location', 'offsite', 'position', 'java', 'caspnet',
       'python', 'php', 'sql', 'angular', 'vue', 'firebase', 'aws',
       'dockerkubernetes', 'design', 'motivation', 'coachability', 'teamwork'],
      dtype='object')
Index(['english', 'spanish', 'portuguese', 'french', 'dutch', 'catalan',
       'location', 'offsite', 'position', 'java', 'caspnet', 'python', 'php',
       'sql', 'angular', 'vue', 'firebase', 'aws', 'dockerkubernetes',
       'design', 'motivation', 'coachability', 'teamwork'],
      dtype='object')


,english,spanish,portuguese,french,dutch,catalan,location,offsite,position,java,...,sql,angular,vue,firebase,aws,dockerkubernetes,design,motivation,coachability,teamwork
name,,,,,,,,,,,,,,,,,,,,,
VT-LAB,4,5,1,1,1,1,1,1,7,2,...,2,2,1,1,1,2,1,3,2,1
Influencity,4,5,1,1,1,1,1,2,5,1,...,1,1,1,1,2,1,1,2,1,3
NEORIS,2,5,1,1,1,1,1,2,5,1,...,1,2,2,1,1,1,1,1,3,2
BICG,3,4,1,1,1,1,1,2,4,1,...,3,2,3,3,2,2,2,2,3,1
ADIDEA,3,5,1,1,1,1,1,1,1,3,...,1,2,1,2,2,2,1,1,3,2


In [4]:
# list(companies_enc.loc['Company ALL MAX']), list(companies_enc.loc['Company ALL MIN'])

In [5]:
# STUDENTS_ENCUESTAS_CSV = './encuestas/Web - Ironhack Matchmaking Student Form (Responses) - Form Responses 1.csv'
# BNC web students day1
# STUDENTS_ENCUESTAS_CSV = './encuestas/BCN - Web Abril - Ironhack Matchmaking Student Form (Responses) - 11APR.csv'

# BNC web students day2
# STUDENTS_ENCUESTAS_CSV = './encuestas/BCN - Web Abril - Ironhack Matchmaking Student Form (Responses) - Form Responses 1.csv'

# Madrid web students
STUDENTS_ENCUESTAS_CSV = './encuestas/HIRING FAIR APR 2019 MATCHMAKING - STUDENTS WD HF APR 2019.csv'

DIC_STUDENTS = {
    "What is your full name?": "name", 
    'What is your level in the following languages? [English]': 'english', 
    'What is your level in the following languages? [Spanish]': 'spanish', 
    'What is your level in the following languages? [Portuguese]': 'portuguese', 
    'What is your level in the following languages? [French]': 'french', 
    'What is your level in the following languages? [Dutch]': 'dutch', 
    'What is your level in the following languages? [Catalan]': 'catalan', 
    'What are your preferences for job location?': 'location', 
    'What is your preference in terms of remote working?': 'offsite', 
    'In you role, how would you prefer to allocate your time between working on backend and working on frontend?': 'position', 
    'Aside from the ones covered in our curriculum (MERN Stack), do you have experience with additional specific hard-skills? [Java]': 'java', 
    'Aside from the ones covered in our curriculum (MERN Stack), do you have experience with additional specific hard-skills? [C# / ASP.net]': 'caspnet',
    'Aside from the ones covered in our curriculum (MERN Stack), do you have experience with additional specific hard-skills? [Python]': 'python',
    'Aside from the ones covered in our curriculum (MERN Stack), do you have experience with additional specific hard-skills? [PHP]': 'php',
    'Aside from the ones covered in our curriculum (MERN Stack), do you have experience with additional specific hard-skills? [SQL]': 'sql',
    'Aside from the ones covered in our curriculum (MERN Stack), do you have experience with additional specific hard-skills? [Angular (Typescript generation)]': 'angular',
    'Aside from the ones covered in our curriculum (MERN Stack), do you have experience with additional specific hard-skills? [VueJS]': 'vue',
    'Aside from the ones covered in our curriculum (MERN Stack), do you have experience with additional specific hard-skills? [Firebase]': 'firebase',
    'Aside from the ones covered in our curriculum (MERN Stack), do you have experience with additional specific hard-skills? [AWS]': 'aws',
    'Aside from the ones covered in our curriculum (MERN Stack), do you have experience with additional specific hard-skills? [Docker / Kubernetes]': 'dockerkubernetes',
    'Aside from the ones covered in our curriculum (MERN Stack), do you have experience with additional specific hard-skills? [Design Tools (Photoshop, Illustrator...)]': 'design', # esta es nueva
    "From this list of soft skills, please rank in order the ones you consider you're stronger at [Motivation and ability to overcome problems: I am passionate, have a can-do attitude and proactively look for solutions to every problem.]": 'motivation',
    "From this list of soft skills, please rank in order the ones you consider you're stronger at [Coachability: I am receptive and actively listen and act on the feedback I receive]": 'coachability',
    "From this list of soft skills, please rank in order the ones you consider you're stronger at [Teamwork: I'm able to receive and provide value in a team environment]\r": 'teamwork' # El \r del final es para que sirva de terminador
    }

students_enc = translate_csv(path=STUDENTS_ENCUESTAS_CSV, dic=DIC_STUDENTS)

Index(['name', 'english', 'spanish', 'portuguese', 'french', 'dutch',
       'catalan', 'location', 'offsite', 'position', 'java', 'caspnet',
       'python', 'php', 'sql', 'angular', 'vue', 'firebase', 'aws',
       'dockerkubernetes', 'design', 'motivation', 'coachability',
       'From this list of soft skills, please rank in order the ones you consider you're stronger at [Teamwork: I'm able to receive and provide value in a team environment]'],
      dtype='object')
Index(['english', 'spanish', 'portuguese', 'french', 'dutch', 'catalan',
       'location', 'offsite', 'position', 'java', 'caspnet', 'python', 'php',
       'sql', 'angular', 'vue', 'firebase', 'aws', 'dockerkubernetes',
       'design', 'motivation', 'coachability',
       'From this list of soft skills, please rank in order the ones you consider you're stronger at [Teamwork: I'm able to receive and provide value in a team environment]'],
      dtype='object')


In [6]:
# students_enc.drop(columns=['todrop'], axis=1, inplace=True)
display(students_enc.head(5))
# display(students_enc.dtypes)
STUDENTS_CLEAN_CSV = './encuestas/students.csv'
students_enc.to_csv(STUDENTS_CLEAN_CSV, index=True)

# students_enc.columns

,english,spanish,portuguese,french,dutch,catalan,location,offsite,position,java,...,sql,angular,vue,firebase,aws,dockerkubernetes,design,motivation,coachability,"From this list of soft skills, please rank in order the ones you consider you're stronger at [Teamwork: I'm able to receive and provide value in a team environment]"
name,,,,,,,,,,,,,,,,,,,,,
Maria Cristina Blanco,5,5,1,2,1,1,3,2,9,1,...,1,1,1,1,1,1,3,1,2,3
Eugenio Ruiz Cepeda,2,5,1,1,1,1,1,2,5,1,...,1,1,1,1,1,1,2,3,2,1
Moisés García Almeida,3,5,1,1,1,3,2,2,4,1,...,3,1,1,2,2,1,2,2,3,1
Nira Alonso Rodríguez,3,5,2,1,1,1,1,2,5,1,...,1,1,1,1,1,1,2,3,2,1
Nerea Aparicio,3,5,1,1,1,1,2,2,5,1,...,1,1,1,1,1,1,2,1,3,2


In [7]:
# list(students_enc.loc['Student ALL MAX']), list(students_enc.loc['Student ALL MIN'])

In [8]:
# Testing
companies_enc.columns == students_enc.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False])

## Tratado de los CSVs

In [9]:
# Alumnos

STUDENTS_CSV = STUDENTS_CLEAN_CSV # antes -> './input/students.csv'
COMPANIES_CSV = COMPANIES_CLEAN_CSV # antes -> './input/companies.csv'

def csv2dataframe(path): 
    return pd.read_csv(path, sep=',', index_col=0, lineterminator='\n')

def adquisition(): 
    students = csv2dataframe(STUDENTS_CSV)
    companies = csv2dataframe(COMPANIES_CSV)
    return students, companies

students, companies = adquisition()

display(len(students.columns), len(companies.columns))

23

23

In [10]:
students.dtypes

english                                                                                                                                                                 int64
spanish                                                                                                                                                                 int64
portuguese                                                                                                                                                              int64
french                                                                                                                                                                  int64
dutch                                                                                                                                                                   int64
catalan                                                                                                                           

In [11]:
companies.dtypes

english             int64
spanish             int64
portuguese          int64
french              int64
dutch               int64
catalan             int64
location            int64
offsite             int64
position            int64
java                int64
caspnet             int64
python              int64
php                 int64
sql                 int64
angular             int64
vue                 int64
firebase            int64
aws                 int64
dockerkubernetes    int64
design              int64
motivation          int64
coachability        int64
teamwork            int64
dtype: object

In [12]:
# normalizacion. Por columnas en alumnos o empresas hay un 0.0 y un 1.0

def normalize_2dfs(students, companies): 
    
    # min(s, c, 0) -> para que no de NaN
    # mins = [min(s, c, 0) for s, c in zip(students.min(), companies.min())]
    print(len(students.min()), students.min(), len(companies.min()), companies.min())
    
    mins = [min(s, c) for s, c in zip(students.min(), companies.min())]
    students = students-mins
    companies = companies-mins
    
    # maxs = [max(s, c, 1) for s, c in zip(students.max(), companies.max())]
    maxs = [max(s, c) for s, c in zip(students.max(), companies.max())]
    students = students/maxs
    companies = companies/maxs
    
    return students, companies

In [13]:
display(students.shape, companies.shape)
students_norm, companies_norm = normalize_2dfs(students, companies)

(30, 23)

(28, 23)

23 english                                                                                                                                                                 2
spanish                                                                                                                                                                 2
portuguese                                                                                                                                                              1
french                                                                                                                                                                  1
dutch                                                                                                                                                                   1
catalan                                                                                                                                            

In [14]:
display(students), display(companies)
display(students_norm), display(companies_norm)

,english,spanish,portuguese,french,dutch,catalan,location,offsite,position,java,...,sql,angular,vue,firebase,aws,dockerkubernetes,design,motivation,coachability,"From this list of soft skills, please rank in order the ones you consider you're stronger at [Teamwork: I'm able to receive and provide value in a team environment]"
name,,,,,,,,,,,,,,,,,,,,,
Maria Cristina Blanco,5,5,1,2,1,1,3,2,9,1,...,1,1,1,1,1,1,3,1,2,3
Eugenio Ruiz Cepeda,2,5,1,1,1,1,1,2,5,1,...,1,1,1,1,1,1,2,3,2,1
Moisés García Almeida,3,5,1,1,1,3,2,2,4,1,...,3,1,1,2,2,1,2,2,3,1
Nira Alonso Rodríguez,3,5,2,1,1,1,1,2,5,1,...,1,1,1,1,1,1,2,3,2,1
Nerea Aparicio,3,5,1,1,1,1,2,2,5,1,...,1,1,1,1,1,1,2,1,3,2
Francisco Leston Ramos,5,5,3,1,1,2,3,1,5,1,...,2,1,1,2,1,1,3,1,2,3
Sergio Ortega Gil,3,5,1,1,1,1,1,2,8,2,...,2,1,1,1,1,1,3,1,2,3
Mario Quiroga Cuervo,5,5,1,2,1,1,3,2,7,1,...,1,2,1,1,2,1,2,1,3,2
Daniel García Martínez,3,5,1,1,1,2,2,2,6,1,...,1,1,1,1,1,1,2,1,2,3


,english,spanish,portuguese,french,dutch,catalan,location,offsite,position,java,...,sql,angular,vue,firebase,aws,dockerkubernetes,design,motivation,coachability,teamwork
name,,,,,,,,,,,,,,,,,,,,,
VT-LAB,4,5,1,1,1,1,1,1,7,2,...,2,2,1,1,1,2,1,3,2,1
Influencity,4,5,1,1,1,1,1,2,5,1,...,1,1,1,1,2,1,1,2,1,3
NEORIS,2,5,1,1,1,1,1,2,5,1,...,1,2,2,1,1,1,1,1,3,2
BICG,3,4,1,1,1,1,1,2,4,1,...,3,2,3,3,2,2,2,2,3,1
ADIDEA,3,5,1,1,1,1,1,1,1,3,...,1,2,1,2,2,2,1,1,3,2
Claire Joster,1,5,1,1,1,1,1,2,4,3,...,3,2,2,1,3,2,1,1,2,3
Cires21,4,4,1,1,1,1,1,2,8,1,...,2,2,3,1,1,2,2,1,3,2
Decathlon,4,5,1,1,1,1,1,2,5,1,...,2,2,2,1,2,1,1,1,3,2
Dhr Global Human Resources S. L.,3,5,1,1,1,1,1,1,5,3,...,1,2,1,1,1,2,1,2,3,1


,english,spanish,portuguese,french,dutch,catalan,location,offsite,position,java,...,sql,angular,vue,firebase,aws,dockerkubernetes,design,motivation,coachability,"From this list of soft skills, please rank in order the ones you consider you're stronger at [Teamwork: I'm able to receive and provide value in a team environment]"
name,,,,,,,,,,,,,,,,,,,,,
Maria Cristina Blanco,1.00,1.000000,0.00,0.5,0.0,0.00,1.0,0.333333,0.888889,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.5,1.0
Eugenio Ruiz Cepeda,0.25,1.000000,0.00,0.0,0.0,0.00,0.0,0.333333,0.444444,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.5,1.0,0.5,0.0
Moisés García Almeida,0.50,1.000000,0.00,0.0,0.0,0.50,0.5,0.333333,0.333333,0.0,...,1.0,0.0,0.0,0.5,0.5,0.0,0.5,0.5,1.0,0.0
Nira Alonso Rodríguez,0.50,1.000000,0.25,0.0,0.0,0.00,0.0,0.333333,0.444444,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.5,1.0,0.5,0.0
Nerea Aparicio,0.50,1.000000,0.00,0.0,0.0,0.00,0.5,0.333333,0.444444,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,1.0,0.5
Francisco Leston Ramos,1.00,1.000000,0.50,0.0,0.0,0.25,1.0,0.000000,0.444444,0.0,...,0.5,0.0,0.0,0.5,0.0,0.0,1.0,0.0,0.5,1.0
Sergio Ortega Gil,0.50,1.000000,0.00,0.0,0.0,0.00,0.0,0.333333,0.777778,0.5,...,0.5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.5,1.0
Mario Quiroga Cuervo,1.00,1.000000,0.00,0.5,0.0,0.00,1.0,0.333333,0.666667,0.0,...,0.0,0.5,0.0,0.0,0.5,0.0,0.5,0.0,1.0,0.5
Daniel García Martínez,0.50,1.000000,0.00,0.0,0.0,0.25,0.5,0.333333,0.555556,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.5,1.0


,english,spanish,portuguese,french,dutch,catalan,location,offsite,position,java,...,sql,angular,vue,firebase,aws,dockerkubernetes,design,motivation,coachability,teamwork
name,,,,,,,,,,,,,,,,,,,,,
VT-LAB,0.75,1.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.666667,0.5,...,0.5,0.5,0.0,0.0,0.0,0.5,0.0,1.0,0.5,0.0
Influencity,0.75,1.000000,0.00,0.0,0.0,0.0,0.0,0.333333,0.444444,0.0,...,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.5,0.0,1.0
NEORIS,0.25,1.000000,0.00,0.0,0.0,0.0,0.0,0.333333,0.444444,0.0,...,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,1.0,0.5
BICG,0.50,0.666667,0.00,0.0,0.0,0.0,0.0,0.333333,0.333333,0.0,...,1.0,0.5,1.0,1.0,0.5,0.5,0.5,0.5,1.0,0.0
ADIDEA,0.50,1.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,1.0,...,0.0,0.5,0.0,0.5,0.5,0.5,0.0,0.0,1.0,0.5
Claire Joster,0.00,1.000000,0.00,0.0,0.0,0.0,0.0,0.333333,0.333333,1.0,...,1.0,0.5,0.5,0.0,1.0,0.5,0.0,0.0,0.5,1.0
Cires21,0.75,0.666667,0.00,0.0,0.0,0.0,0.0,0.333333,0.777778,0.0,...,0.5,0.5,1.0,0.0,0.0,0.5,0.5,0.0,1.0,0.5
Decathlon,0.75,1.000000,0.00,0.0,0.0,0.0,0.0,0.333333,0.444444,0.0,...,0.5,0.5,0.5,0.0,0.5,0.0,0.0,0.0,1.0,0.5
Dhr Global Human Resources S. L.,0.50,1.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.444444,1.0,...,0.0,0.5,0.0,0.0,0.0,0.5,0.0,0.5,1.0,0.0


(None, None)

In [15]:
students_norm.fillna(0, inplace=True)
companies_norm.fillna(0, inplace=True)

In [16]:
students_norm.dtypes

english                                                                                                                                                                 float64
spanish                                                                                                                                                                 float64
portuguese                                                                                                                                                              float64
french                                                                                                                                                                  float64
dutch                                                                                                                                                                   float64
catalan                                                                                                                 

In [17]:
companies_norm.dtypes

english             float64
spanish             float64
portuguese          float64
french              float64
dutch               float64
catalan             float64
location            float64
offsite             float64
position            float64
java                float64
caspnet             float64
python              float64
php                 float64
sql                 float64
angular             float64
vue                 float64
firebase            float64
aws                 float64
dockerkubernetes    float64
design              float64
motivation          float64
coachability        float64
teamwork            float64
dtype: object

In [18]:
# Matching

# PESOS = (alumnos.columns)

'''
P = [2,1,1,1,3,1,1,1,1,5, 1, 1, 1, 1, 1, 1, 1 ,1, 1, 1, 1, 1 ,1, 1, 1]
W = {col: p for col, p in zip(students_norm.columns, P)}
'''

PESOS = {
    'english': 1,
    'spanish': 1,
    'portuguese': 1,
    'french': 1,
    'dutch': 1,
    'catalan': 1,
    'location': 1, 
    'offsite': 1,
    'position': 5, # importante
    'java': 5,     # importante las técnicas
    'caspnet': 5,
    'python': 5,
    'php': 5,
    'sql': 5,
    'angular': 5,
    'vue': 5,
    'firebase': 5,
    'aws': 5,
    'dockerkubernetes': 5,
    'design': 5,
    'motivation': 1,
    'coachability': 1,
    'teamwork': 1
}


def calc_dist(s_array, c_array, weights_val): 
    ''' 
    https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.euclidean.html
    
    https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cosine.html
    scipy.spatial.distance.cosine
    '''
    return calc_match(distance.euclidean(s_array, c_array, weights_val))

def calc_match(distance): 
    ''' [0, 1] -> [1, 0]'''
    return 1/(1+distance)
'''
def match(student, company, weights=W): 
    return calc_dist(student, company, list(weights.values()))
'''
def match(student, company, weights=PESOS): 
    student = [min(s, c) for s, c in zip(student, company)] # añadimos para que la sobrecualificación de un estudiante no empeore el matchin
    return calc_dist(student, company, list(weights.values()))

In [19]:
# match(students_norm.loc['alumno1'], companies_norm.loc['empresa1'])


# Emparejamiento

En cada ronda se crea una pareja alumno-empresa empezando por el mejor %. Luego se pasa al siguente alumno. Si la empresa que hacía mejor match es la del caso anterior, pasa a su segundo mejor match, etc. 

En cada ronda se cambia el estudiante que elige primero, entre los que no han ido primeros

In [20]:
# !jupyter kernelspec list
# import sys
# sys.path

In [21]:
# create dataframe de tamañ0 SxC

matching = pd.DataFrame(np.zeros((len(students), len(companies))) , columns=companies.index, index=students.index)
# matching.head()

In [22]:
# matching.index, matching.columns
students_norm.dtypes

english                                                                                                                                                                 float64
spanish                                                                                                                                                                 float64
portuguese                                                                                                                                                              float64
french                                                                                                                                                                  float64
dutch                                                                                                                                                                   float64
catalan                                                                                                                 

In [23]:
companies_norm.dtypes

english             float64
spanish             float64
portuguese          float64
french              float64
dutch               float64
catalan             float64
location            float64
offsite             float64
position            float64
java                float64
caspnet             float64
python              float64
php                 float64
sql                 float64
angular             float64
vue                 float64
firebase            float64
aws                 float64
dockerkubernetes    float64
design              float64
motivation          float64
coachability        float64
teamwork            float64
dtype: object

In [24]:
# display(students_norm.head()), display(companies_norm.head())

In [25]:
# matching.loc['alumno1', 'empresa3']

def calculate_match(matching, df1, df2): 
    # matching = pd.DataFrame(np.zeros((len(students), len(companies))) , columns=companies.index, index=students.index)
    for a in matching.index: 
        print('index', a)
        for c in matching.columns:
            print('column', c)
            try: 
                matching.loc[a, c] = match(df1.loc[a], df2.loc[c], weights=PESOS)
            except: 
                matching.loc[a, c] = 0
    return matching

matching = calculate_match(matching, students_norm, companies_norm)

# matching es una matriz de elementos entre [0, 1]
# matching.min().min(), matching.max().max()

matching

index Maria Cristina Blanco
column VT-LAB
column Influencity
column NEORIS
column BICG
column ADIDEA
column Claire Joster
column Cires21
column Decathlon
column Dhr Global Human Resources S. L.
column Mintyhost
column Optimissa Capital Markets
column BMind
column Winning Scientific Management 
column Sopra Steria
column BinPar Team SL
column UXSmobile
column Page Personnel
column Sipay Plus
column Minsait 
column Excibit Solutions Spain
column Deloitte
column Flexicar
column BuscoExtra
column Alterna
column The Phone House 
column Kabel
column Chiara
column Digital Talent Agency 
index Eugenio Ruiz Cepeda
column VT-LAB
column Influencity
column NEORIS
column BICG
column ADIDEA
column Claire Joster
column Cires21
column Decathlon
column Dhr Global Human Resources S. L.
column Mintyhost
column Optimissa Capital Markets
column BMind
column Winning Scientific Management 
column Sopra Steria
column BinPar Team SL
column UXSmobile
column Page Personnel
column Sipay Plus
column Minsait 
colum

column BMind
column Winning Scientific Management 
column Sopra Steria
column BinPar Team SL
column UXSmobile
column Page Personnel
column Sipay Plus
column Minsait 
column Excibit Solutions Spain
column Deloitte
column Flexicar
column BuscoExtra
column Alterna
column The Phone House 
column Kabel
column Chiara
column Digital Talent Agency 
index Rodrigo Otero Montero
column VT-LAB
column Influencity
column NEORIS
column BICG
column ADIDEA
column Claire Joster
column Cires21
column Decathlon
column Dhr Global Human Resources S. L.
column Mintyhost
column Optimissa Capital Markets
column BMind
column Winning Scientific Management 
column Sopra Steria
column BinPar Team SL
column UXSmobile
column Page Personnel
column Sipay Plus
column Minsait 
column Excibit Solutions Spain
column Deloitte
column Flexicar
column BuscoExtra
column Alterna
column The Phone House 
column Kabel
column Chiara
column Digital Talent Agency 
index Verónica Celemín Lominchar
column VT-LAB
column Influencity
colu

name,VT-LAB,Influencity,NEORIS,BICG,ADIDEA,Claire Joster,Cires21,Decathlon,Dhr Global Human Resources S. L.,Mintyhost,...,Minsait,Excibit Solutions Spain,Deloitte,Flexicar,BuscoExtra,Alterna,The Phone House,Kabel,Chiara,Digital Talent Agency
name,,,,,,,,,,,,,,,,,,,,,
Maria Cristina Blanco,0.222222,0.449490,0.376179,0.168793,0.227730,0.187613,0.238007,0.264279,0.202552,0.281729,...,0.169521,0.191825,0.154934,0.225844,0.314520,1.000000,0.255397,0.183685,0.200000,0.225844
Eugenio Ruiz Cepeda,0.225867,0.387426,0.366025,0.181589,0.225381,0.183685,0.229233,0.274292,0.210526,0.277002,...,0.171767,0.193771,0.158945,0.238007,0.363899,0.500000,0.266667,0.186359,0.229179,0.238007
Moisés García Almeida,0.232738,0.485367,0.373577,0.212400,0.250000,0.231662,0.239797,0.301365,0.204878,0.333333,...,0.181364,0.202243,0.196956,0.284712,0.309281,0.500000,0.302598,0.209083,0.238007,0.284712
Nira Alonso Rodríguez,0.227277,0.396717,0.366025,0.169521,0.225844,0.183685,0.230714,0.260454,0.203868,0.277926,...,0.168612,0.189160,0.156471,0.227730,0.325205,0.500000,0.267479,0.187110,0.205213,0.227730
Nerea Aparicio,0.220074,0.426205,0.387426,0.169521,0.229668,0.186605,0.234822,0.266667,0.203868,0.285714,...,0.170074,0.192377,0.154797,0.227730,0.302720,0.666667,0.255397,0.183216,0.200000,0.227730
Francisco Leston Ramos,0.240281,0.440665,0.371543,0.195839,0.250000,0.204612,0.243361,0.280017,0.225844,0.301622,...,0.181412,0.203541,0.188180,0.259875,0.300537,1.000000,0.288013,0.221014,0.205973,0.259875
Sergio Ortega Gil,0.288832,0.444444,0.376179,0.180907,0.264279,0.229668,0.263499,0.302582,0.235828,0.333333,...,0.191553,0.202227,0.173913,0.326632,0.348331,1.000000,0.348331,0.206241,0.200000,0.326632
Mario Quiroga Cuervo,0.231662,0.585786,0.472136,0.177405,0.252650,0.210897,0.251987,0.309017,0.210897,0.340542,...,0.187613,0.220481,0.167366,0.227730,0.346350,0.666667,0.255397,0.200000,0.231662,0.227730
Daniel García Martínez,0.221351,0.444444,0.376179,0.168793,0.227730,0.187613,0.235320,0.263499,0.202552,0.281729,...,0.169338,0.191553,0.154797,0.225844,0.309083,1.000000,0.255397,0.183448,0.200000,0.225844


In [26]:
# create csv de pruebas
'''
df_pruebas = matching.copy()     
df1 = df_pruebas.iloc[:, :15].copy() # 30x15
df2 = df_pruebas.head(15).copy()     # 15x30
df3 = df_pruebas.copy()              # 30x30

df1.shape, df2.shape, df3.shape
'''
pass

# Grafos

In [27]:
import networkx as nx
import matplotlib

# https://networkx.github.io/documentation/networkx-1.10/reference/introduction.html

def create_graph(df): 
    G_res=nx.Graph()
    for a in df.index: 
        for e in df.columns: 
            G_res.add_edge(a,e,weight=df.loc[a][e])
    return G_res

def plot_bipartite_graph(G, set_X, set_Y): 
    X, Y = set_X, set_Y # df1.index, df1.columns
    pos = dict()
    pos.update( (n, (1, i)) for i, n in enumerate(X) ) # put nodes from X at x=1
    pos.update( (n, (2, i)) for i, n in enumerate(Y) ) # put nodes from Y at x=2
    nx.draw(G, pos=pos)

    '''
def get_best_match2(G, node): 
    # sort por peso DESCENCENTE
    for edge in sorted(G.edges(data=True), key=lambda x: - x[2]['weight']):
        if node in edge: 
            return edge
'''    
# sort por peso DESCENCENTE

def get_best_match(G, node, used): 
    # sort por peso DESCENCENTE
    for edge in sorted(G.edges(data=True), key=lambda x: - x[2]['weight']):
        if node in edge and all(u not in edge for u in used): 
            # return edge
            res = edge
            break
    else: 
        # elegimos uno random??
        options = [(node1, node2, w) for node1, node2, w in list(G.edges(data=True)) if node1 not in used and node2 not in used]
        aleatorio = choice(options)
        # print(aleatorio)
        # print("No hay emparejamiento posible") 
        # print(used)
        
        # return aleatorio
        res = aleatorio
    
    node1, node2, w = res
    return node1, node2, w['weight']
        
 
    


# favoreciendo empresas. más alumnos que empresas 

In [28]:
# node1, node2, w

# df_pruebas = matching.copy()

In [29]:
'''
G = create_graph(df_pruebas) # G = create_graph(df1)
students = df_pruebas.index
companies = df_pruebas.columns

plot_bipartite_graph(G, students, companies)

node1, node2, w = get_best_match(G, 'BMind', [])
node1, node2, w

'''

"\nG = create_graph(df_pruebas) # G = create_graph(df1)\nstudents = df_pruebas.index\ncompanies = df_pruebas.columns\n\nplot_bipartite_graph(G, students, companies)\n\nnode1, node2, w = get_best_match(G, 'BMind', [])\nnode1, node2, w\n\n"

In [30]:
# companies

In [31]:
'''
RONDAS = 3 #10
MESAS = min(len(students), len(companies))
MESAS

rondas = []

for ronda in range(RONDAS): 
    aux = []
    used = []
    for company in companies: 
        
        node1, node2, w = get_best_match(G, company, used)
        # aux.append((node1, node2, w))
        G.remove_edge(node1, node2)
        used.extend([node1, node2])
        if node1 == company: 
            # used.append(node2)
            aux.append((node1, node2, w))
        else: 
            # used.append(node1)
            aux.append((node2, node1, w))
    rondas.append(aux)
    print(rondas)
    
# rondas
'''
pass

# favorecemos a los estudiantes. más alumnos que empresas

In [32]:
df_pruebas = matching.copy()     

G = create_graph(df_pruebas)
students_list = df_pruebas.index
companies_list = df_pruebas.columns

'''
plot_bipartite_graph(G, students_list, companies_list)

node1, node2, w = get_best_match(G, 'Company ALL MID', [])
node1, node2, w
'''

"\nplot_bipartite_graph(G, students_list, companies_list)\n\nnode1, node2, w = get_best_match(G, 'Company ALL MID', [])\nnode1, node2, w\n"

In [33]:
matching

name,VT-LAB,Influencity,NEORIS,BICG,ADIDEA,Claire Joster,Cires21,Decathlon,Dhr Global Human Resources S. L.,Mintyhost,...,Minsait,Excibit Solutions Spain,Deloitte,Flexicar,BuscoExtra,Alterna,The Phone House,Kabel,Chiara,Digital Talent Agency
name,,,,,,,,,,,,,,,,,,,,,
Maria Cristina Blanco,0.222222,0.449490,0.376179,0.168793,0.227730,0.187613,0.238007,0.264279,0.202552,0.281729,...,0.169521,0.191825,0.154934,0.225844,0.314520,1.000000,0.255397,0.183685,0.200000,0.225844
Eugenio Ruiz Cepeda,0.225867,0.387426,0.366025,0.181589,0.225381,0.183685,0.229233,0.274292,0.210526,0.277002,...,0.171767,0.193771,0.158945,0.238007,0.363899,0.500000,0.266667,0.186359,0.229179,0.238007
Moisés García Almeida,0.232738,0.485367,0.373577,0.212400,0.250000,0.231662,0.239797,0.301365,0.204878,0.333333,...,0.181364,0.202243,0.196956,0.284712,0.309281,0.500000,0.302598,0.209083,0.238007,0.284712
Nira Alonso Rodríguez,0.227277,0.396717,0.366025,0.169521,0.225844,0.183685,0.230714,0.260454,0.203868,0.277926,...,0.168612,0.189160,0.156471,0.227730,0.325205,0.500000,0.267479,0.187110,0.205213,0.227730
Nerea Aparicio,0.220074,0.426205,0.387426,0.169521,0.229668,0.186605,0.234822,0.266667,0.203868,0.285714,...,0.170074,0.192377,0.154797,0.227730,0.302720,0.666667,0.255397,0.183216,0.200000,0.227730
Francisco Leston Ramos,0.240281,0.440665,0.371543,0.195839,0.250000,0.204612,0.243361,0.280017,0.225844,0.301622,...,0.181412,0.203541,0.188180,0.259875,0.300537,1.000000,0.288013,0.221014,0.205973,0.259875
Sergio Ortega Gil,0.288832,0.444444,0.376179,0.180907,0.264279,0.229668,0.263499,0.302582,0.235828,0.333333,...,0.191553,0.202227,0.173913,0.326632,0.348331,1.000000,0.348331,0.206241,0.200000,0.326632
Mario Quiroga Cuervo,0.231662,0.585786,0.472136,0.177405,0.252650,0.210897,0.251987,0.309017,0.210897,0.340542,...,0.187613,0.220481,0.167366,0.227730,0.346350,0.666667,0.255397,0.200000,0.231662,0.227730
Daniel García Martínez,0.221351,0.444444,0.376179,0.168793,0.227730,0.187613,0.235320,0.263499,0.202552,0.281729,...,0.169338,0.191553,0.154797,0.225844,0.309083,1.000000,0.255397,0.183448,0.200000,0.225844


In [34]:
from random import shuffle

RONDAS = 11
# print(RONDAS)

# print(len(students_list), students_list)

# student_queue = list(students_list) * RONDAS # vamos a recorrer la lista para ir emparejando

lista = list(students_list)
shuffle(lista)
# print(lista)
student_queue = lista * RONDAS # vamos a recorrer la lista para ir emparejando
# print(student_queue)

In [35]:
rondas = []

for r in range(RONDAS): 
    aux = []
    used = []
    while len(used) < 2 * len(companies): # ahora used tiene el doble de tamaño. Hay que modificarlo
        student = student_queue.pop(0)
        # print(student)
        node1, node2, w = get_best_match(G, student, used) # creo que se queda sin opciones
        
        # aux.append((node1, node2, w))
        G.remove_edge(node1, node2)
        used.extend([node1, node2])
        
    
        # el nodo1 es la compañía (nodo1 no es estudiante)
        if node1 != student: 
            # used.append(node1)
            aux.append((node1, node2, w))
        # nodo2 es la compañía
        else: 
            # used.append(node2)
            aux.append((node2, node1, w))
        # print(used)
    rondas.append(aux)
        
rondas

[[('Alterna', 'Javier Requejo Melchor', 1.0),
  ('Influencity', 'Daniel Herrero Hernando', 0.5865374495209118),
  ('BinPar Team SL', 'Verónica Celemín Lominchar', 0.6666666666666666),
  ('NEORIS', 'Javier Pérez-Chao Rodríguez', 0.44948974278317805),
  ('BuscoExtra', 'Carlos Eduardo Lores Hernández ', 0.2942980550855494),
  ('Mintyhost', 'Patricia Defez Díaz de Sarralde', 0.30383243470068705),
  ('Decathlon', 'Alfonso Silva Feria', 0.28884555672059276),
  ('The Phone House ', 'Carlos Martínez Crespo', 0.3463498301839386),
  ('Flexicar', 'Francisco Leston Ramos', 0.25987517556175926),
  ('Cires21', 'Alfredo Hinarejos Corbeá', 0.237455908152084),
  ('VT-LAB', 'Thilo Sperber', 0.2272595996093377),
  ('Digital Talent Agency ', 'Eugenio Ruiz Cepeda', 0.23800671553691075),
  ('ADIDEA', 'Daniel García Martínez', 0.22772999919644132),
  ('Page Personnel', 'Moisés García Almeida', 0.24935973861346583),
  ('Dhr Global Human Resources S. L.',
   'Rodrigo Otero Montero',
   0.2658725560274332),
  (

In [36]:
def check_rondas(rondas): 
    res = []
    for ronda in rondas: 
        companies = [company for company, student, w in ronda]
        students = [student for company, student, w in ronda]
        res.append(len(set(companies)) == len(set(students)) )
    return all(e for e in res)
            
check_rondas(rondas)            

True

In [37]:
def rondas2string(rondas):
    res = ''
    for i, ronda in enumerate(rondas): 
        res += '\n\nRonda {}'.format(i)
        for c, s, w in ronda: 
            res += '\nCompany: {} -> Student: {} -> Matching: {}'.format(c, s, w)
        
    return res
        
        
text = rondas2string(rondas)

with open("Rondas.txt", "w") as f:
    f.write(text)

In [38]:
# beautifultable

table = BeautifulTable()
table.column_headers = ["ronda", "company", "student", "match"]
for i, ronda in enumerate(rondas): 
    for row in ronda: 
        table.append_row([i, row[0], row[1], row[2]])
        
# print(table)

with open("Rondas_table.txt", "w") as f:
    f.write(str(table))

In [39]:
# agrupadas por compañía: 

table = BeautifulTable()
table.column_headers = ["company", "ronda", "student", "match"]

for company in companies_list: 
    for i, ronda in enumerate(rondas): 
        for row in ronda: 
            if row[0] == company: 
                table.append_row([row[0], i, row[1], row[2]])
        
# print(table)

with open("Rondas_table_by_company.txt", "w") as f:
    f.write(str(table))

In [40]:
companies_list

Index(['VT-LAB', 'Influencity', 'NEORIS', 'BICG', 'ADIDEA', 'Claire Joster',
       'Cires21', 'Decathlon', 'Dhr Global Human Resources S. L.', 'Mintyhost',
       'Optimissa Capital Markets', 'BMind', 'Winning Scientific Management ',
       'Sopra Steria', 'BinPar Team SL', 'UXSmobile', 'Page Personnel',
       'Sipay Plus', 'Minsait ', 'Excibit Solutions Spain', 'Deloitte',
       'Flexicar', 'BuscoExtra', 'Alterna', 'The Phone House ', 'Kabel',
       'Chiara', 'Digital Talent Agency '],
      dtype='object', name='name')

In [41]:

table = BeautifulTable()
table.column_headers = ["student", "company", "ronda", "match"]

for student in students_list: 
    for i, ronda in enumerate(rondas): 
        for row in ronda: 
            if row[1] == student: 
                table.append_row([row[1], row[0], i, row[2]])
'''

'''    
with open("Rondas_table_by_student.txt", "w") as f:
    f.write(str(table))

In [42]:
student_rounds = {}

for student in students_list: 
    total = 0
    for ronda in rondas: 
        for row in ronda: 
            if row[1] == student: 
                total += 1
    student_rounds[student] = total

student_rounds

{'Maria Cristina Blanco': 10,
 'Eugenio Ruiz Cepeda': 10,
 'Moisés García Almeida': 10,
 'Nira Alonso Rodríguez': 10,
 'Nerea Aparicio': 10,
 'Francisco Leston Ramos': 10,
 'Sergio Ortega Gil': 10,
 'Mario Quiroga Cuervo': 10,
 'Daniel García Martínez': 10,
 'Carlos Eduardo Lores Hernández ': 11,
 'Javier Requejo Melchor': 11,
 'Enrique Anaya Marin': 10,
 'Maria Isabel Llanes Agudelo': 10,
 'Daniel Herrero Hernando': 11,
 'Thilo Sperber': 10,
 'Carlos Martínez Crespo': 11,
 'Dan Mihai Grigoriciuc': 10,
 'Rodrigo Otero Montero': 10,
 'Verónica Celemín Lominchar': 11,
 'Abel Colmenares Napolitano': 10,
 'Javier Pérez-Chao Rodríguez': 11,
 'Jaime Jacobo Rodríguez del Valle': 10,
 'Patricia Defez Díaz de Sarralde': 11,
 'Isabel Rodríguez Godoy': 10,
 'Sofia Jordan': 10,
 'Alfonso Silva Feria': 11,
 'Alfredo Hinarejos Corbeá': 10,
 'Baltasar Torres Garcia': 10,
 'FAKE1': 10,
 'FAKE2': 10}

In [43]:
print(min(student_rounds.values()), max(student_rounds.values()))

10 11
